# Predict on new data using a trained CNN on XPS data on Google Colab

In this notebook, we will use a trained convolutional network to predict on unseen XPS spectra.

## Setup

### Mount google drive, change working directory

In [ ]:
# Mount drive
from google.colab import drive
import os

drive.mount('/content/drive')

# Change working path
os.chdir('/content/drive/My Drive/deepxps')

### Install packages and import modules

In [ ]:
%%capture
# Install packages
!pip install python-docx
!pip install xlsxwriter
# Import standard modules and magic commands
import datetime
import numpy as np
import pytz
import importlib
import matplotlib.pyplot as plt

# Magic commands
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Disable tf warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf

### Check TensorFlow version

In [ ]:
tf.__version__

### Load custom modules

In [ ]:
try:
    import importlib
    importlib.reload(classifier)
    importlib.reload(clfutils)
    print('\n Modules were reloaded.')
except:
    import xpsdeeplearning.network.classifier as classifier
    import xpsdeeplearning.network.utils as clfutils
    print('Modules were loaded.')

### Set up the parameters & folder structure

In [ ]:
np.random.seed(502)
time = datetime.datetime.now().astimezone(pytz.timezone('Europe/Berlin')).strftime("%Y%m%d_%Hh%Mm")
exp_name = "Ni_2_classes_linear_comb_regressiob_MC_dropout_predict_using_20220215_20h15m"

clf = classifier.Classifier(time = time,
                            exp_name = exp_name,
                            task = "regression",
                            intensity_only = True)

### Load and inspect the data

In [ ]:
input_filepath = r'/content/drive/My Drive/deepxps/datasets/20210528_Ni_linear_combination_small_gas_phase_detection_only.h5'

train_test_split = 0.99
train_val_split = 0
no_of_examples = 1000

        
X_train, X_val, X_test, y_train, y_val, y_test,\
    names_train, names_val, names_test =\
        clf.load_data_preprocess(input_filepath = input_filepath,
                                 no_of_examples = no_of_examples,
                                 train_test_split = train_test_split,
                                 train_val_split = train_val_split)
        
# Check how the examples are distributed across the classes.
class_distribution = clf.datahandler.check_class_distribution(clf.task)
clf.plot_class_distribution()
clf.plot_random(no_of_spectra = 10, dataset = 'test')  

### Load and compile the model

In [ ]:
from tensorflow.python.keras import backend as K
clf.load_model(model_path = '/content/drive/My Drive/deepxps/runs/20220215_20h15m_Ni_2_classes_linear_comb_small_gas_phase_multi_class_detection_CNN_MC_dropout/model')

### Plot summary and save model plot.


In [ ]:
clf.summary()
clf.save_and_print_model_image()

## Predict on current data set

### Evaluate on test data

In [ ]:
clf.logging.hyperparams['batch_size'] = 32

if clf.task == 'classification':
    score = clf.evaluate()
    test_loss, test_accuracy = score[0], score[1]
    print('Test loss: ' + str(np.round(test_loss, decimals=8)))
    print('Test accuracy: ' + str(np.round(test_accuracy, decimals=3)))
elif clf.task == 'regression':
    test_loss = clf.evaluate()
    print('Test loss: ' + str(np.round(test_loss, decimals=8)))

### Show predictions with current model

In [ ]:
no_of_predictions = 100

prob_pred_train = clf.predict_probabilistic(
    dataset="train",
    no_of_predictions=no_of_predictions
)
prob_pred_test = clf.predict_probabilistic(
    dataset="test",
    no_of_predictions=no_of_predictions
)

for i, pred in enumerate(prob_pred_test[:10]):
   print(f"Ground truth: {np.round(clf.datahandler.y_test[i],3)},",
         f"Mean prediction: {np.mean(pred, axis = 0)} +/- {np.std(pred, axis = 0)}")

### Show some predictions on random test samples

In [ ]:
clf.plot_prob_predictions(dataset="test",
                          no_of_spectra=20,
                          to_file=True)

## Create out-of-scope dataset

### Plot specific spectra from classifier

In [ ]:
indices = np.arange(10)
no_of_spectra = len(indices)

clf.plot_spectra_by_indices(
    indices=indices, 
    dataset="test",
    with_prediction=True)

### Create synthetic spectra

In [ ]:
from xpsdeeplearning.simulation.base_model.spectra import SyntheticSpectrum
from xpsdeeplearning.simulation.base_model.peaks import Gauss

def calc_step_size(x_array):
    return np.round(x_array[0] - x_array[1], 2)

def create_flat_line(energies, X_real):
    syn_spectrum = SyntheticSpectrum(
        energies[-1],
        energies[0], 
        step=calc_step_size(energies),
        label="flat"
        )
    syn_spectrum.lineshape += np.min(X_real)
    syn_spectrum.normalize()
    np.expand_dims(syn_spectrum.lineshape, axis=1)
    syn_spectrum.lineshape = syn_spectrum.lineshape[:, None]  

    return syn_spectrum.lineshape

def create_synthetic_peaks(energies, X_real):
    syn_spectrum = SyntheticSpectrum(
        energies[-1],
        energies[0], 
        step=calc_step_size(energies),
        label="peaks"
        )
    syn_spectrum.add_component(
        Gauss(position=855, width=5, intensity=0.0025)
        )
    syn_spectrum.add_component(
        Gauss(position=875, width=5, intensity=0.0025)
        )
    syn_spectrum.lineshape += np.min(X_real)
    syn_spectrum.normalize()
    syn_spectrum.lineshape = syn_spectrum.lineshape[:, None]  

    return syn_spectrum.lineshape

def create_synthetic_peaks_with_slope(energies, X_real):
    syn_spectrum = SyntheticSpectrum(
        energies[-1],
        energies[0], 
        step=calc_step_size(energies),
        label="peaks_with_slope"
        )
    syn_spectrum.add_component(
        Gauss(position=855, width=5, intensity=0.005)
        )
    syn_spectrum.add_component(
        Gauss(position=875, width=5, intensity=0.005)
        )
    syn_spectrum.lineshape += np.min(X_real)
    syn_spectrum.normalize()
    slope = float((X_real[0] - X_real[-1]) / (energies[0] - energies[-1]))
    intercept = X_real[0] - slope*energies[0]
    line = np.array([(slope*x + intercept) for x in energies])
    syn_spectrum.lineshape = syn_spectrum.lineshape[:, None]  
    syn_spectrum.lineshape = np.add(syn_spectrum.lineshape, line)
    syn_spectrum.lineshape -= np.average(X_real)

    return syn_spectrum.lineshape

def test_plot_spectra(index):
    energies = clf.datahandler.energies
    X_real = clf.datahandler.X_test[index]

    fig, ax = plt.subplots(
        nrows=1, ncols=1, figsize=(10,10)
    )
    ax.plot(energies, X_real)
    ax.plot(energies, X_syn_line, linewidth=5)
    ax.plot(energies, X_syn_peaks, linewidth=5)
    ax.plot(energies, X_syn_slope, linewidth=5)
    ax.invert_xaxis()
    ax.set_xlim(np.max(energies), np.min(energies))

    fontdict = {"size": 20}
    ax.set_xlabel("Binding energy (eV)",
                   fontdict=fontdict)
    ax.set_ylabel("Intensity (arb. units)", 
                   fontdict=fontdict)
    ax.tick_params(axis="x", labelsize=fontdict["size"])
    ax.set_yticklabels([])

index = 9 # Change index of the baseline spectrum here.

X_syn_line = create_flat_line(
    energies=clf.datahandler.energies,
    X_real=clf.datahandler.X_test[index])
X_syn_peaks = create_synthetic_peaks(
    energies=clf.datahandler.energies,
    X_real=clf.datahandler.X_test[index])
X_syn_slope = create_synthetic_peaks_with_slope(
    energies=clf.datahandler.energies,
    X_real=clf.datahandler.X_test[index])

X_syn = np.hstack((X_syn_line,X_syn_peaks,X_syn_slope))
X_syn = X_syn[None, :, :].transpose(2, 1, 0)
test_plot_spectra(index=index)

In [ ]:
no_of_predictions = 100
y_syn = np.zeros([X_syn.shape[0], len(clf.datahandler.labels)])
prob_pred_syn = np.array(
    [clf.model.predict(X_syn) for i in range(no_of_predictions)]
    ).transpose(1, 0, 2)
print("Mean prediction:")
for i, pred in enumerate(prob_pred_syn[:10]):
    print(f"{np.mean(pred, axis = 0)} +/- {np.std(pred, axis = 0)}")

### Plot probabilistic predictions individually

In [ ]:
def plot_prob_predictions_individual(no_of_real_spectra=5):
    random_numbers = []
    for i in range(no_of_real_spectra):
        random_numbers.append(np.random.randint(0, clf.datahandler.X_test.shape[0]))
    
    X = np.vstack((clf.datahandler.X_test[random_numbers], X_syn))
    y = np.vstack((clf.datahandler.y_test[random_numbers], y_syn))
    prob_pred = np.vstack((clf.datahandler.prob_pred_test[random_numbers], prob_pred_syn))

    energies = clf.datahandler.energies

    fontdict = {"size": 15}
    fig, axs = plt.subplots(
        nrows=X.shape[0], 
        ncols=len(clf.datahandler.labels)+1,
        )
    plt.subplots_adjust(
        left=0.125,
          bottom=0.5,
        right=4.8,
        top=X.shape[0],
        wspace=0.4,
        hspace=0.4,
        )
    
    cmap = plt.get_cmap("tab10")

    for i in range(X.shape[0]):
        color = cmap(0) if i < no_of_real_spectra else cmap(1)

        for j, label in enumerate(clf.datahandler.labels):
            axs[i,j+1].set_title(label,fontdict=fontdict)
            axs[i,j+1].set_xlabel("Prediction",fontdict=fontdict)
            axs[i,j+1].set_ylabel("Counts",fontdict=fontdict)
            axs[i,j+1].tick_params(axis="x", labelsize=fontdict["size"])
            axs[i,j+1].tick_params(axis="y", labelsize=fontdict["size"])

        axs[i, 0].plot(energies, X[i],color=color)
        axs[i, 0].invert_xaxis()
        axs[i, 0].set_xlim(np.max(energies), np.min(energies))
        axs[i, 0].set_xlabel("Binding energy (eV)", fontdict=fontdict)
        axs[i, 0].set_ylabel("Intensity (arb. units)", fontdict=fontdict)
        axs[i,0].tick_params(axis="x", labelsize=fontdict["size"])
        axs[i,0].set_yticks([])


        for j, row in enumerate(prob_pred[i].transpose()):
            # This is to show that it works, should be removed later.
            if i >= no_of_real_spectra:
                row = np.random.normal(np.mean(row),scale=0.1,size=prob_pred.shape[1])
            axs[i,j+1].hist(
                row,
                bins=50,
                range=(0.,1.),
                orientation="vertical",
                color=color,
                fill=True,
                linewidth=1,
                label=clf.datahandler.labels[j],
            )
            axs[i,j+1].set_ylim([0, prob_pred.shape[1]])

    fig.tight_layout()

    return fig

fig = plot_prob_predictions_individual(no_of_real_spectra=5)

### Plot probabilistic predictions together

In [ ]:
def plot_prob_predictions_together(no_of_real_spectra=2):
    random_numbers = []
    for i in range(no_of_real_spectra):
        random_numbers.append(np.random.randint(0, clf.datahandler.X_test.shape[0]))
    
    X = np.vstack((clf.datahandler.X_test[random_numbers], X_syn))
    X.shape
    y = np.vstack((clf.datahandler.y_test[random_numbers], y_syn))
    prob_pred = np.vstack((clf.datahandler.prob_pred_test[random_numbers], prob_pred_syn))

    energies = clf.datahandler.energies

    fontdict = {"size": 15}
    fig, axs = plt.subplots(
        nrows=1, 
        ncols=len(clf.datahandler.labels)+1,
        )
    plt.subplots_adjust(
        #left=0.025,
        right=2,
        wspace=0.5)

    for j, label in enumerate(clf.datahandler.labels):
        axs[j+1].set_title(label,fontdict=fontdict)
        axs[j+1].set_xlabel("Prediction",fontdict=fontdict)
        axs[j+1].set_ylabel("Counts",fontdict=fontdict)
        axs[j+1].tick_params(axis="x", labelsize=fontdict["size"])
        axs[j+1].tick_params(axis="y", labelsize=fontdict["size"])

    cmap = plt.get_cmap("tab10")
    for i in range(X.shape[0]):
        color = cmap(i)

        axs[0].plot(energies, X[i],color=color)
        axs[0].invert_xaxis()
        axs[0].set_xlim(np.max(energies), np.min(energies))
        axs[0].set_xlabel("Binding energy (eV)", fontdict=fontdict)
        axs[0].set_ylabel("Intensity (arb. units)", fontdict=fontdict)
        axs[0].tick_params(axis="x", labelsize=fontdict["size"])
        axs[0].set_yticks([])

        for j, row in enumerate(prob_pred[i].transpose()):
            # This is to show that it works, should be removed later.
            if i >= no_of_real_spectra:
                row = np.random.normal(np.mean(row),scale=0.1,size=prob_pred.shape[1])
            axs[j+1].hist(
                row,
                bins=50,
                range=(0.,1.),
                orientation="vertical",
                color=color,
                fill=True,
                linewidth=1,
                label=clf.datahandler.labels[j],
            )
            axs[j+1].set_ylim([0, prob_pred.shape[1]])

   # fig.tight_layout()

    return fig

fig = plot_prob_predictions_together(no_of_real_spectra=1)

### Save data

In [ ]:
#clf.save_hyperparams()
clf.pickle_results()

## Save output of notebook

In [ ]:
from IPython.display import Javascript, display
from nbconvert import HTMLExporter

def save_notebook():
    display(Javascript("IPython.notebook.save_notebook()"),
            include=['application/javascript'])

def output_HTML(read_file, output_file):
    import codecs
    import nbformat
    exporter = HTMLExporter()
    # read_file is '.ipynb', output_file is '.html'
    output_notebook = nbformat.read(read_file, as_version=4)
    output, resources = exporter.from_notebook_node(output_notebook)
    codecs.open(output_file, 'w', encoding='utf-8').write(output)

import time
import os

time.sleep(20)
save_notebook()
print('Notebook saved!')
time.sleep(30)
current_file = '/content/drive/My Drive/deepxps/xpsdeeplearning/notebooks/predict_bayesian.ipynb'
output_file = os.path.join(clf.logging.log_dir,
                           'predict_bayesian_out.html')
output_HTML(current_file, output_file)
print('HTML file saved!')